# Cluster the Spire
Will Wright

### Purpose and Context

[todo]

In [143]:
# Load packages
import shutil
from os import listdir
import json
import glob
import os
import numpy as np
import pandas as pd

# increase viewable dataframe rows and columns
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 20)

All the data currently lives in several zipped tar.gz files within the 'zipped' folder.  These need to be extracted into an unzipped folder.

**PROTIP:** If you have the files already extracted (as they are in the repo), skip this step to avoid the lengthy unpacking process

In [2]:
def extract_all(archives, extract_path, zip_format = "gztar"):
    '''
    input: path to zipped file archives, path to extract, and type of zipped file
    output: unzipped contents of each zipped file within the extract path
    '''
    for filename in listdir(archives):
        shutil.unpack_archive(archives+filename, extract_path, zip_format)

In [32]:
extract_all("../data_raw/zipped/","../data_raw/unzipped/", "gztar")

In [3]:
# Start here if the files are already unzipped
read_files = glob.glob("../data_raw/unzipped/*/*.json", recursive = True)

To give more context about the data we're working with, lets see exactly how many raw game runs we have:

In [4]:
len(read_files)

279848

Almost 280K games! We'll need to subset down to games for The Defect on Ascension 20 that resulted in wins before we can determine the relevant sample size though. In order to do that, we'll want to read these files together and use relevant JSON keys to narrow our focus.

In [5]:
# this approach creates a list of JSON strings from all the read_files
output_list = []

for f in read_files:
    try:
        with open(f, "r") as infile:
            # test if the file isn't empty and that the name doesn't contain 'undefined' (1 file, contents are "File doesn't exists)")
            if (os.path.getsize(f)>0) & (('undefined' in f)==False):
                output_list.append(json.load(infile))
            else:
                pass
    except UnicodeDecodeError: # some unicode can't be read so just don't load those games (I think it's a particular monster name)
        pass
    

In [6]:
len(output_list)

279693

In [7]:
len(read_files)-len(output_list)

155

We've excluded 155 games that were either empty, had unreadable unicode, or were 'undefined'.  It's possible that this may introduce some bias (e.g. removing relevant games with particular qualities), but given that this represents such a small volume of games relative to all 280K and I haven't seen any apparent bias in looking through a sample of the files, I don't think this should be a major concern.

After more attempts to get the data into the right format, it looks like there is a single case where the JSON is wrapped in '[ ]'.  Since this game is for Ironclad, I'll simply remove from the dataset.

In [8]:
len(output_list)

279693

In [9]:
output_list[:] = [s for s in output_list if str(s)[0]!='[']

In [10]:
len(output_list)

279692

With that single exception removed, we can now subset to a list of Defect games, which pass the conditions of being the Defect character, a victory, and Ascension 20.  Since it's possible that I'll want to expand this investigation to the other two characters later, I'll also set aside their games in their own lists.

In [25]:
# Winning Ascension 20 games by character
defect_asc20_win_games = []
ironclad_asc20_win_games = []
silent_asc20_win_games = []

# Losing Ascension 20 games by character
defect_asc20_lose_games = []
ironclad_asc20_lose_games = []
silent_asc20_lose_games = []

for i in range(len(output_list)):
    if output_list[i] is not None:
        # test to ensure the game data has all the required elements (character, ascention level, and victory status)
        if ('character_chosen' in dict(output_list[i])) and \
        ('ascension_level' in dict(output_list[i])) and \
        ('victory' in dict(output_list[i])):
            
            # DEFECT WINNING
            if (output_list[i]['character_chosen']=='DEFECT') & \
            (output_list[i]['victory']==True) & \
            (output_list[i]['ascension_level']==20):
                defect_asc20_win_games.append(output_list[i])
            
            # DEFECT LOSING
            if (output_list[i]['character_chosen']=='DEFECT') & \
            (output_list[i]['victory']==False) & \
            (output_list[i]['ascension_level']==20):
                defect_asc20_lose_games.append(output_list[i])
            
            # IRONCLAD WINNING  
            if (output_list[i]['character_chosen']=='IRONCLAD') & \
            (output_list[i]['victory']==True) & \
            (output_list[i]['ascension_level']==20):
                ironclad_asc20_win_games.append(output_list[i])
                
            # IRONCLAD LOSING  
            if (output_list[i]['character_chosen']=='IRONCLAD') & \
            (output_list[i]['victory']==False) & \
            (output_list[i]['ascension_level']==20):
                ironclad_asc20_lose_games.append(output_list[i])
                
            # SILENT WINNING  
            if (output_list[i]['character_chosen']=='THE_SILENT') & \
            (output_list[i]['victory']==True) & \
            (output_list[i]['ascension_level']==20):
                silent_asc20_win_games.append(output_list[i])
                
            # SILENT LOSING
            if (output_list[i]['character_chosen']=='THE_SILENT') & \
            (output_list[i]['victory']==False) & \
            (output_list[i]['ascension_level']==20):
                silent_asc20_lose_games.append(output_list[i])

I'm curious about character winrates.  Lets compare to the total games per character.

In [45]:
# Calculate all summary statistics
defect_winning = len(defect_asc20_win_games)
defect_losing = len(defect_asc20_lose_games)
defect_total = len(defect_asc20_win_games)+len(defect_asc20_lose_games)
defect_winrate = defect_winning/defect_total

# Calculate all summary statistics
ironclad_winning = len(ironclad_asc20_win_games)
ironclad_losing = len(ironclad_asc20_lose_games)
ironclad_total = len(ironclad_asc20_win_games)+len(ironclad_asc20_lose_games)
ironclad_winrate = ironclad_winning/ironclad_total

# Calculate all summary statistics
silent_winning = len(silent_asc20_win_games)
silent_losing = len(silent_asc20_lose_games)
silent_total = len(silent_asc20_win_games)+len(silent_asc20_lose_games)
silent_winrate = silent_winning/silent_total


asc20_games_summary = pd.DataFrame({'Character':['Defect','Ironclad','Silent'],
                                    'Winning Games':[defect_winning,
                                                     ironclad_winning,
                                                     silent_winning],
                                     'Total Games':[defect_total,
                                                    ironclad_total,
                                                    silent_total],
                                     'Winrate':[defect_winrate,
                                               ironclad_winrate,
                                               silent_winrate]})



In [46]:
asc20_games_summary

,Character,Winning Games,Total Games,Winrate
0,Defect,1669,16863,0.098974
1,Ironclad,1716,14798,0.115962
2,Silent,1811,14278,0.126838


It would seem that although Defect is the most-played character, it has the lowest winrate.  This supports the claim that Defect is the hardest character (at least on Ascension 20).  In any case, we have 1669 victorious Defect Ascension 20 games, which should be adequate sample for clustering.

Next, we need to convert this list of JSON objects to a dataframe we can cluster.  Ideally, the shape of the data is one-row-per-game with columns for all the cards and relics. In order to do that, we'll want to create a vector of all unique cards and relics.  

#### Getting Unique Cards and Relics  

In order to get all unique cards and relics, we can simply pull all cards and relics from all games, then apply the `unique()` function.

In [57]:
all_game_decks = []
all_game_relics = []

for i in range(len(output_list)):
    if output_list[i] is not None:
        # ensure the run data has the deck and relics to avoid errors in rare cases
        if ('master_deck' in dict(output_list[i])) and ('relics' in dict(output_list[i])):
            all_game_decks.append(output_list[i]['master_deck'])
            all_game_relics.append(output_list[i]['relics'])

In [66]:
# Within each game, each card and relic needs to be pulled out into a flat list.
all_cards = []

for i in range(len(all_game_decks)):
    for j in range(len(all_game_decks[i])):
        all_cards.append(all_game_decks[i][j])
        
all_relics = []

for i in range(len(all_game_relics)):
    for j in range(len(all_game_relics[i])):
        all_relics.append(all_game_relics[i][j])


In [67]:
# create unique lists
unique_cards = list(np.unique(all_cards))
unique_relics = list(np.unique(all_relics))

In [68]:
len(unique_cards)

3164

In [69]:
len(unique_relics)

876

Looks like we have 3164 unique cards and 876 unique relics.  This is a fair bit more than expected, so lets take a look at the head and tail of cards:

In [78]:
unique_cards[0:10]

['6A',
 '6A+1',
 'A Thousand Cuts',
 'A Thousand Cuts+1',
 'Abandon',
 'Abandon+1',
 'AbeCurse',
 'AbsoluteMagnitude+1',
 'Absolvement',
 'Absolvement+1']

In [79]:
unique_cards[-10:-1]

['vexMod:StarBlast',
 'vexMod:StrikeStorm',
 'vexMod:StrikeStorm+1',
 'vexMod:Taunt+1',
 'vexMod:TrainingStrike',
 'vexMod:TrainingStrike+1',
 'vexMod:UltimateCard',
 'vexMod:VenomSigh',
 'vexMod:VolumeVengeance']

This reveals two issues: there are the standard and "+1" versions of cards (players can upgrade cards once) as well as cards from game mods (essentially, player-made extensions of the game).  Thankfully, my domain expertise makes it fairly easy to know which cards aren't in the base game and it seems like most of the modded cards have a ':' in their name so they should be fairly easy to exclude.  

After testing, it looks like there are a few other exceptions for specific mods that use a '\_' in their name.  I'll go ahead and simply remove those cases as well.

In [88]:
unique_cards[:] = [s for s in unique_cards if '+' not in s \
                   and ':' not in s\
                   and '_' not in s]

In [90]:
len(unique_cards)

716

In review of the new card list, I can still see some non-base cards, but I'm not too concerned with this affecting the final results due to the expected low frequency of those cards (0 in cases where the character isn't one of the base characters).  

Next, the same cleansing will be applied to the relics. Generally speaking, relics have the same issue with mods as the cards, but there are not upgrades available.

In [92]:
unique_relics[:] = [s for s in unique_relics if '_' not in s \
                   and ':' not in s]

In [93]:
len(unique_relics)

391

Again, this isn't a perfect methodology, but since there are no flags for being a mod within the game data, it is difficult to use a single signal as a subsetting criteria to only the base game.

___

At this point, we can build a table of all unique cards and relics and then fill in Trues and Falses for whether the card was present per completed game.

In [130]:
def resource_table_generator(resource_input, game_input):
    '''
    input: a list of resources to be included in the rows of the table (i.e. cards and relics)
           a list of JSON game data to go in the columns
    output: a DataFrame indicating if each resource is in each game with a True or False
    '''
    # Build a 1-column table with the resources in each row
    resource_table = pd.DataFrame({'Resource':resource_input})
    
    # determine the number of preceding zeroes to put in the game name for the column names
    game_volume = len(game_input)
    preceding_zeroes = len(str(game_volume)) # so if you have 1669 games, the first game will be 'game_0001'
    
    # For each game append a column and fill with True/False based on if the card is present
    for i in range(len(game_input)):
        # Create column name for the game
        resource_table['game_'+str('{:0'+str(preceding_zeroes)+'d}').format(i+1)] = False # default to false
        # For each resource, update to True if it is in the final game resources
        for j in range(len(resource_table)):
            if((resource_table['Resource'][j] in game_input[i]['master_deck']) or \
               (resource_table['Resource'][j] in game_input[i]['relics'])):
                resource_table.iloc[j,i+1] = True
    
    return(resource_table)


Now, simply plug in the resources and the Defect Ascension 20 victorious games to get a complete table of whether each resourece is present in each game.

In [154]:
defect_asc20_win_resources = resource_table_generator(unique_cards+unique_relics, defect_asc20_win_games)

In [155]:
defect_asc20_win_resources

,Resource,game_0001,game_0002,game_0003,game_0004,game_0005,game_0006,game_0007,game_0008,game_0009,...,game_1660,game_1661,game_1662,game_1663,game_1664,game_1665,game_1666,game_1667,game_1668,game_1669
0,6A,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,A Thousand Cuts,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,Abandon,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,AbeCurse,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Absolvement,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1102,Winged Necklace,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1103,WingedGreaves,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1104,WristBlade,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1105,Yang,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


It'd be useful to see the relative frequency of each resource in a separate summary table:

In [195]:
defect_resource_freq = pd.DataFrame({'Resource':defect_asc20_win_resources['Resource'],
                                     'Frequency':defect_asc20_win_resources.sum(axis = 1)})
defect_resource_freq = defect_resource_freq.sort_values(by = ['Frequency'], ascending = False).reset_index(drop=True)
defect_resource_freq['Percent of Wins'] = defect_resource_freq['Frequency']/len(defect_asc20_win_games)

In [210]:
# subset to resources which are >0
defect_resource_freq = defect_resource_freq[defect_resource_freq['Frequency']>0]
defect_resource_freq['Percentile Rank']=defect_resource_freq['Frequency'].rank(pct=True)
defect_resource_freq['Rank']=defect_resource_freq['Frequency'].rank(ascending = False)

In [214]:
defect_resource_freq

,Resource,Frequency,Percent of Wins,Percentile Rank,Rank
0,AscendersBane,1646.0,0.986219,1.000000,1.0
1,Cracked Core,1394.0,0.835231,0.997319,2.0
2,Defend_B,978.0,0.585980,0.994638,3.0
3,Dualcast,823.0,0.493110,0.991957,4.0
4,Glacier,719.0,0.430797,0.989276,5.0
...,...,...,...,...,...
368,Corruption,1.0,0.000599,0.093834,339.0
369,HandOfGreed,1.0,0.000599,0.093834,339.0
370,Snack Pack,1.0,0.000599,0.093834,339.0
371,Ninja Scroll,1.0,0.000599,0.093834,339.0


These results mostly align with expectations.  'AscendersBane' is the curse card which should be present in every winning deck with Defect on Ascension 20 and thus, it makes sense that it's the most frequent card (though I'm not quire sure how some games completed without it).  Below that is the starting relic 'Cracked Core' so it makes sense that that's there too.  In fact, the first resource that isn't a starting resource is 'Glacier', making it the highest-rated resource to get. Despite it's desirability, only 43% of wins have the card and it's likely that the cluster analysis will reveal the decks where this card (and all cards) are better or worse, based on the rest of the resources.

In [197]:
# Silent 
silent_asc20_win_resources = resource_table_generator(unique_cards+unique_relics, silent_asc20_win_games)
silent_resource_freq = pd.DataFrame({'Resource':silent_asc20_win_resources['Resource'],
                                     'Frequency':silent_asc20_win_resources.sum(axis = 1)})
silent_resource_freq = silent_resource_freq.sort_values(by = ['Frequency'], ascending = False).reset_index(drop=True)
silent_resource_freq['Percent of Wins'] = silent_resource_freq['Frequency']/len(silent_asc20_win_games)

,Resource,Frequency,Percent of Wins
0,AscendersBane,1785,0.985643
1,Ring of the Snake,1493,0.824406
2,Survivor,1207,0.666483
3,Defend_G,1049,0.579238
4,Backflip,794,0.438432
...,...,...,...
195,Flash of Steel,44,0.024296
196,Toy Ornithopter,43,0.023744
197,Nilry's Codex,41,0.022639
198,Accuracy,41,0.022639


In [212]:
# subset to resources which are >0
silent_resource_freq = silent_resource_freq[silent_resource_freq['Frequency']>0]
silent_resource_freq['Percentile Rank']=silent_resource_freq['Frequency'].rank(pct=True)
silent_resource_freq['Rank']=silent_resource_freq['Frequency'].rank(ascending = False)

In [213]:
silent_resource_freq[0:100]

,Resource,Frequency,Percent of Wins,Percentile Rank,Rank
0,AscendersBane,1785,0.985643,1.000000,1.0
1,Ring of the Snake,1493,0.824406,0.997984,2.0
2,Survivor,1207,0.666483,0.995968,3.0
3,Defend_G,1049,0.579238,0.993952,4.0
4,Backflip,794,0.438432,0.991935,5.0
5,PiercingWail,644,0.355605,0.989919,6.0
6,Oddly Smooth Stone,546,0.301491,0.987903,7.0
7,Bag of Preparation,532,0.293760,0.985887,8.0
8,NeowsBlessing,484,0.267256,0.983871,9.0
9,Snake Skull,482,0.266151,0.981855,10.0


In [ ]:
# Ironclad
ironclad_asc20_win_resources = resource_table_generator(unique_cards+unique_relics, ironclad_asc20_win_games)
ironclad_resource_freq = pd.DataFrame({'Resource':ironclad_asc20_win_resources['Resource'],
                                     'Frequency':ironclad_asc20_win_resources.sum(axis = 1)})
ironclad_resource_freq = ironclad_resource_freq.sort_values(by = ['Frequency'], ascending = False).reset_index(drop=True)
ironclad_resource_freq['Percent of Wins'] = ironclad_resource_freq['Frequency']/len(ironclad_asc20_win_games)

In [207]:
# subset to resources which are >0
ironclad_resource_freq = defect_resource_freq[defect_resource_freq['Frequency']>0]
ironclad_resource_freq['Percentile Rank']=ironclad_resource_freq['Frequency'].rank(pct=True)
ironclad_resource_freq['Rank']=ironclad_resource_freq['Frequency'].rank(ascending = False)

In [208]:
ironclad_resource_freq[0:100]

,Resource,Frequency,Percent of Wins,Percentile Rank,Rank
0,AscendersBane,1682,0.980186,1.000000,1.0
1,Burning Blood,1554,0.905594,0.999097,2.0
2,Defend_R,1218,0.709790,0.998193,3.0
3,Shrug It Off,1017,0.592657,0.997290,4.0
4,Strike_R,639,0.372378,0.996387,5.0
5,Bag of Preparation,558,0.325175,0.995483,6.0
6,Reaper,545,0.317599,0.994580,7.0
7,Oddly Smooth Stone,527,0.307110,0.993677,8.0
8,Bash,519,0.302448,0.992773,9.0
9,Impervious,498,0.290210,0.991870,10.0


In [209]:
# Write to .csv
defect_resource_freq.to_csv('../results/defect_resource_freq.csv', index = False)
ironclad_resource_freq.to_csv('../results/ironclad_resource_freq.csv', index = False)
silent_resource_freq.to_csv('../results/silent_resource_freq.csv', index = False)

___
### Clustering
And now, back to the clustering.